<a href="https://colab.research.google.com/github/RudinP/SOLUX_Summer_data_study/blob/master/4%EC%A3%BC%EC%B0%A8/%EC%84%A4%EB%AC%B8%EC%A1%B0%EC%82%AC_%EC%9D%91%EB%8B%B5_%EB%B6%84%EC%84%9D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#matplotlib의 subplots()사용법, seaborn으로 count,box,distplot 그려보기
#mcq 파일이 너무 커서... 코드를 따라 치기만 했다.

In [ ]:
#블로그,팟캐스트,수업,기타 등등 추천할 만한 것
mcq['BlogsPodcastsNewslettersSelect'] = mcq[
    'BlogsPodcastsNewslettersSelect'
].astype('str').apply(lambda x: x.split(','))#여러 항목이 있어 split, 하나의 row로
mcq['BlogsPodcastsNewslettersSelect'].head()

In [ ]:
s = mcq.apply(lambda x: pd.Series(x['BlogsPodcastsNewslettersSelect']),
              axis=1).stack().reset_index(level=1, drop=True)
s.name = 'platforms'
s.head()


In [ ]:
s = s[s != 'nan'].value_counts().head(20)#상위 20개의 값만 찍어보도록 함

plt.figure(figsize=(6,8))
plt.title("Most Popular Blogs and Podcasts")
sns.barplot(y=s.index, x=s)

In [ ]:
mcq['CoursePlatformSelect'] = mcq[
    'CoursePlatformSelect'].astype(
    'str').apply(lambda x: x.split(','))
mcq['CoursePlatformSelect'].head()

In [ ]:
t = mcq.apply(lambda x: pd.Series(x['CoursePlatformSelect']),
              axis=1).stack().reset_index(level=1, drop=True)
t.name = 'courses'
t.head(20)

In [ ]:
t = t[t != 'nan'].value_counts()

In [ ]:
plt.title("Most Popular Course Platforms")
sns.barplot(y=t.index, x=t)#그래프 그리기 

In [ ]:
#데이터 사이언스 직무에서 가장 중요하다고 생각되는 스킬
job_features = [
    x for x in mcq.columns if x.find(
        'JobSkillImportance') != -1 
    and x.find('JobSkillImportanceOther') == -1]#jobskillimportance가 들어간것을 찾음

job_features

In [ ]:
jdf = {}
for feature in job_features:
    a = mcq[feature].value_counts()
    a = a/a.sum()
    jdf[feature[len('JobSkillImportance'):]] = a

jdf

In [ ]:
jdf = pd.DataFrame(jdf).transpose()#transpose를 하면 x축과 y축이 서로 바뀌게 된다
jdf

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(jdf.sort_values("Necessary", 
                            ascending=False), annot=True)#heatmap으로 그래프 그리기(degree에 따라 그려짐)

In [ ]:
jdf.plot(kind='bar', figsize=(12,6), 
         title="Skill Importance in Data Science Jobs")
plt.xticks(rotation=60, ha='right')#barplot을 matplotlib으로 그림.막대그래프 모양

In [ ]:
#데이터과학자의 평균 급여

mcq[mcq['CompensationAmount'].notnull()].shape

In [ ]:
mcq['CompensationAmount'] = mcq[
    'CompensationAmount'].str.replace(',','')
mcq['CompensationAmount'] = mcq[
    'CompensationAmount'].str.replace('-','')#이러한 값들이 있으면 계산에 오류가 나기 때문에, str로 없애줌.

# 환율계산을 위한 정보 가져오기
rates = pd.read_csv('data/conversionRates.csv')#환율 정보 csv
rates.drop('Unnamed: 0',axis=1,inplace=True)

salary = mcq[
    ['CompensationAmount','CompensationCurrency',
     'GenderSelect',
     'Country',
     'CurrentJobTitleSelect']].dropna()
salary = salary.merge(rates,left_on='CompensationCurrency',
                      right_on='originCountry', how='left')
salary['Salary'] = pd.to_numeric(
    salary['CompensationAmount']) * salary['exchangeRate']
salary.head()

In [ ]:
print('Maximum Salary is USD $',
      salary['Salary'].dropna().astype(int).max())
print('Minimum Salary is USD $',
      salary['Salary'].dropna().astype(int).min())
print('Median Salary is USD $',
      salary['Salary'].dropna().astype(int).median())#차례대로 na값은 드롭하고 최대,최소,평균 급여를 보여줌.

In [ ]:
plt.subplots(figsize=(15,8))
salary=salary[salary['Salary']<500000]
sns.distplot(salary['Salary'])
plt.axvline(salary['Salary'].median(), linestyle='dashed')#원하는 값에 선을 그어줄 수 있음.
plt.title('Salary Distribution',size=15)

In [ ]:
plt.subplots(figsize=(8,12))#그래프가 찌그러져 보이면 숫자 조정하면 됨

sal_coun = salary.groupby(
    'Country')['Salary'].median().sort_values(
    ascending=False)[:30].to_frame()

sns.barplot('Salary', 
            sal_coun.index,
            data = sal_coun,
            palette='RdYlGn')

plt.axvline(salary['Salary'].median(), linestyle='dashed')
plt.title('Highest Salary Paying Countries')

In [ ]:
plt.subplots(figsize=(8,4))
sns.boxplot(y='GenderSelect',x='Salary', data=salary)#박스모양 그래프를 그림

In [ ]:
salary_korea = salary.loc[(salary['Country']=='South Korea')]#한국의 데이터만 따로 구해 그래프화
plt.subplots(figsize=(8,4))
sns.boxplot(y='GenderSelect',x='Salary',data=salary_korea)

In [ ]:
salary_korea.shape

In [ ]:
salary_korea_male = salary_korea[
    salary_korea['GenderSelect']== 'Male']
salary_korea_male['Salary'].describe()

In [ ]:
salary_korea_male

In [ ]:
#개인 프로젝트나 학습용 데이터를 얻는 곳
mcq['PublicDatasetsSelect'] = mcq[
    'PublicDatasetsSelect'].astype('str').apply(
    lambda x: x.split(',')
    )

In [ ]:
q = mcq.apply(
    lambda x: pd.Series(x['PublicDatasetsSelect']),
    axis=1).stack().reset_index(level=1, drop=True)

q.name = 'courses'

In [ ]:
q = q[q != 'nan'].value_counts()

In [ ]:
pd.DataFrame(q)

In [ ]:
plt.title("Most Popular Dataset Platforms")
sns.barplot(y=q.index, x=q)

In [ ]:
# 주관식 응답을 읽어온다.
ff = pd.read_csv('data/freeformResponses.csv', 
                 encoding="ISO-8859-1", low_memory=False)
ff.shape

In [ ]:
# 설문내용과 누구에게 물어봤는지를 찾아봄
qc = question.loc[question[
    'Column'].str.contains('PersonalProjectsChallengeFreeForm')]
print(qc.shape)
qc.Question.values[0]

In [ ]:
ppcff = ff[
    'PersonalProjectsChallengeFreeForm'].value_counts().head(15)
ppcff.name = '응답 수'
pd.DataFrame(ppcff)

In [ ]:
#데이터 사이언스 업무에서 가장 많은 시간을 필요로 하는 일은?
time_features = [
    x for x in mcq.columns if x.find('Time') != -1][4:10]

In [ ]:
tdf = {}
for feature in time_features:
    tdf[feature[len('Time'):]] = mcq[feature].mean()

tdf = pd.Series(tdf)
print(tdf)
print()

plt.pie(tdf, labels=tdf.index, 
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title("Percentage of Time Spent on Each DS Job")
plt.show()